# 2-qubit DME

Suppose our target and instruction states involve 2 qubits.
Then $\sigma=\sigma_1\otimes \sigma_2$, $\rho=\rho_1\otimes\rho_2$.
$$
\begin{align}
\text{SWAP}(\sigma\otimes\rho)&&=\text{SWAP}[(\sigma_1\otimes \sigma_2)\otimes(\rho_1\otimes\rho_2)]\\
&&=\text{SWAP}[(\rho_1\otimes\rho_2)\otimes(\sigma_1\otimes \sigma_2)]
\end{align}
$$

Which is equivalent to swapping $\sigma_i$ with $\rho_i$.

In [8]:
# imports
from qiskit import QuantumCircuit
import numpy as np
from utils import *
from qiskit_ionq import IonQProvider
from copy import deepcopy
from utils import *
import qiskit.quantum_info as qi
from scipy.linalg import expm


## Example

Here, we want to show that an entangled 2-qubit state $\sigma=\sigma_1\otimes\sigma_2$, in our example the Bell state $|\varphi\rangle=|00\rangle + |11\rangle$, can be fully swapped with two instruction qubits $\rho_1$ and $\rho_2$ without affecting the correlation.

That is, $\text{SWAP}_2[\text{SWAP}_2(\sigma\otimes\rho)] = \rho\otimes\sigma$

Suppose $\sigma = q_0\otimes q_1 = |00\rang\lang 00|$, $\rho = q_2\otimes q_3=|+\rang\lang+|\otimes|+\rang\lang+|$.


In [9]:
# initialize states matrices for calculation
state_0 = np.outer(np.array([1,0]),np.array([1,0]))
state_p = np.outer(np.array([1,1]),np.array([1,1]))/2
sigma = np.kron(state_p, state_p)
rho = np.kron(state_0, state_0)
dm_tot = np.kron(sigma, rho)

In [10]:
# build circuit to produce the desired state
qc = QuantumCircuit(4,4)
# bell-state sigma
qc.h(0)
qc.cx(0,1)
print(qc.draw())

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
q_2: ──────────
               
q_3: ──────────
               
c: 4/══════════
               


In [11]:
t = np.pi/2
add_partial_swap(qc, 0, 2, t)
add_partial_swap(qc, 1, 3, t)
qc.measure([0,1],[0,1])
qc.measure([2,3], [2,3])
print(qc.draw())

     ┌───┐                ░ ┌───┐┌───┐          ┌───┐┌─────┐┌───┐ ░           »
q_0: ┤ H ├──■───■─────────░─┤ S ├┤ H ├─■────────┤ H ├┤ Sdg ├┤ H ├─░──■────────»
     └───┘┌─┴─┐ │         ░ └───┘└───┘ │        └───┘└─────┘└───┘ ░  │        »
q_1: ─────┤ X ├─┼─────────░────────────┼──────────────────────────░──┼────────»
          └───┘ │ZZ(π/2)  ░ ┌───┐┌───┐ │ZZ(π/2) ┌───┐┌─────┐┌───┐ ░  │ZZ(π/2) »
q_2: ───────────■─────────░─┤ S ├┤ H ├─■────────┤ H ├┤ Sdg ├┤ H ├─░──■────────»
                          ░ └───┘└───┘          └───┘└─────┘└───┘ ░           »
q_3: ─────────────────────░───────────────────────────────────────░───────────»
                          ░                                       ░           »
c: 4/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«       ┌───┐         ░                                       ░           ┌─┐»
«q_0: ──┤ H ├─────────░──────────────────

With $t=\pi/2$, we obtain a full SWAP operation. Therefore, we will expect the outcome of measuring qubit 0,1 to be $|00\rangle$

In [12]:
# setup simulator
my_api_key = "pOiUVlzriOoF2wX1kp3lIqid1OMhwXZ5"
provider = IonQProvider(my_api_key)
simulator_backend = provider.get_backend("ionq_simulator")

In [13]:
# without noise
shots = 1000
job = simulator_backend.run(qc, shots=shots)
result_wo = job.result()
dict_wo = get_keys_freq(result_wo)
print(dict_wo)

{'0000': 0.507, '0011': 0.493}


This proves that our assumption is correct: the correlation between the instruction qubits remain after the swap.